In [1]:
import subprocess
import speech_recognition as sr
import nltk
from nltk import sent_tokenize
import random
import editdistance
import re

#nltk.download('punkt')
#nltk.download('gutenberg')
#print(nltk.corpus.gutenberg.fileids())

In [16]:
p1 = re.compile(r"[^A-Z ']")
p2 = re.compile(r"''")
p3 = re.compile(r'[ ]+')

en_books = [
    ('easy', 'bryant-stories.txt'),
    ('easy', 'burgess-busterbrown.txt'),
    ('medium', 'melville-moby_dick.txt'),
    ('hard', 'chesterton-ball.txt'), 
    ('hard', 'edgeworth-parents.txt'),    
]
# Also see: https://www.nltk.org/book/ch02.html
en_levels = {}

def normalize(sent):
    sent = ' '.join(sent).replace(" ' ", "''")
    sent = sent.upper()
    sent = re.sub(p1, '', sent)
    sent = re.sub(p2, "'", sent)
    sent = re.sub(p3, ' ', sent)
    return sent.strip()

for level, book in en_books:
    lst = en_levels.setdefault(level, [])
    sents = [normalize(sent) for sent in nltk.corpus.gutenberg.sents(book)[2:]]
    lst += list(set([sent for sent in sents if 2 < sent.count(' ') < 10]))

In [18]:
EN_VOICES = ['Alex', 'Victoria', 'Fred', 'Samantha', 'Daniel']

LANG = {
    'en': {'code': 'EN-us', 'voice': EN_VOICES[0], 'levels': en_levels}
}

lang = None
sentences = None

while lang is None:
    options = ", ".join([key for key in LANG.keys()])
    choice = input('Please select language ({}): '.format(options))
    lang = LANG.get(choice)
CODE = lang['code']
VOICE = lang['voice']

while sentences is None:
    options = ", ".join([str(key) for key in lang['levels']])
    choice = input('Please select level ({}): '.format(options))
    sentences = lang['levels'].get(choice)

def say(message):
    subprocess.call(['say', '-v', VOICE, "'{}'".format(message)])

say('Let us begin')

Please select language (en): en
Please select level (easy, medium, hard): medium


In [ ]:
r = sr.Recognizer()
QUESTIONS = 3
score = 0

    
for i in range(QUESTIONS):
    sentence = random.choice(sentences)
    print('Repeat after me...')
    say('Repeat after me')
    print()
    print(sentence.upper())
    say(sentence)
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        print('(start talking)')
        audio = r.listen(source)
        spoken = r.recognize_google(audio, language=lang['code'])
        dist = editdistance.eval(sentence, spoken.upper())
        if dist < 5:
            score += 2
            message = random.choice(['Excellent!', 'Perfect!'])
        elif dist < 10:
            score += 1
            message = random.choice(['You did fine.'])
        else:
            message = random.choice(['OK. Keep going.', 'Keep practicing.'])
        say(message)
        print(message)
        print('You said:', spoken)
        print()

result = 100 * score / (2 * QUESTIONS)
message = 'You scored {}%'.format(int(result))
print(message)
message = 'You scored {} percent'.format(int(result))
say(message)


Repeat after me...

WAS THERE EVER SUCH UNCONSCIOUSNESS
(start talking)
Excellent!
You said: was there ever such unconsciousness

Repeat after me...

WHAT SOULLESS THING IS THIS THAT LAUGHS BEFORE A WRECK
(start talking)
OK. Keep going.
You said: what soulless thing is this bad last before rack

Repeat after me...

THE OIL IN THE HOLD IS LEAKING SIR
(start talking)
